In [ ]:
!pip install pandas openpyxl openai langchain tiktoken langchain-community langchain-openai faiss-cpu
!pip install --upgrade langchain
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Using cached openai-1.91.0-py3-none-any.whl.metadata (26 kB)
Using cached openai-1.91.0-py3-none-any.whl (735 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0


In [ ]:
!pip show langchain openai


Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 
---
Name: openai
Version: 1.86.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
OPENAI_API_KEY="개인키"

In [ ]:
import os
import sqlite3
import pandas as pd
import re

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter

# 파일 경로 설정
EXCEL_DIR = "/content"
DB_PATH = "temp.db"

# 전역 retriever
retriever = None

# 테이블 이름을 SQLite에 안전하게 변환
def sanitize_table_name(name):
    name = os.path.splitext(name)[0]  # 확장자 제거
    name = re.sub(r'[^a-zA-Z0-9_]', '_', name)  # 특수문자 제거
    if name[0].isdigit():
        name = '_' + name  # 숫자로 시작하면 밑줄 추가
    return name

# 1. 엑셀 → SQLite 저장 (여러 파일 처리)
def load_all_excels_to_sqlite(excel_dir, db_path):
    conn = sqlite3.connect(db_path)
    for filename in os.listdir(excel_dir):
        if filename.endswith(".xlsx"):
            filepath = os.path.join(excel_dir, filename)
            table_name = sanitize_table_name(filename)
            df = pd.read_excel(filepath)
            df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

# 2. SQLite에서 모든 테이블 데이터 로드
def fetch_all_dataframes_from_sqlite(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = [row[0] for row in cursor.fetchall()]
    all_dfs = []
    for table_name in table_names:
        df = pd.read_sql_query(f"SELECT * FROM '{table_name}'", conn)
        all_dfs.append(df)
    conn.close()
    return all_dfs

# 3. 데이터프레임 리스트 → LangChain Document 리스트
def dataframes_to_documents(df_list):
    docs = []
    for df in df_list:
        for _, row in df.iterrows():
            content = "\n".join([f"{col}: {row[col]}" for col in df.columns])
            content += "\n\n[주의: 위 내용을 벗어난 정보를 생성하지 말 것. 실제 데이터의 컬럼명과 값만 사용할 것.]"
            docs.append(Document(page_content=content))
    return docs

# 4. RAG 파이프라인 초기화
def initialize_rag():
    global retriever
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY  # 키 설정

    # 엑셀 → SQLite 저장
    load_all_excels_to_sqlite(EXCEL_DIR, DB_PATH)

    # SQLite → Document 리스트
    df_list = fetch_all_dataframes_from_sqlite(DB_PATH)
    docs = dataframes_to_documents(df_list)

    # 문서 쪼개기 + 벡터화
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever()

# 5. 질문에 대해 RAG 실행
def run_query(question: str):
    if retriever is None:
        raise RuntimeError("Retriever is 초기화되지 않았습니다. 먼저 initialize_rag()를 호출하세요.")

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    prompt = ChatPromptTemplate.from_template("""
    아래는 실제 데이터베이스에서 추출한 정보야. 아래 내용만 참고해서 질문에 정확하게 답변해줘.
    허구의 값을 생성하거나 문서에 없는 컬럼명을 추론하지 마.

    1. 질문이 여러 개의 단어로 구성된다면, 공백으로 분할하여 각 단어에 대응하는 컬럼(물리명)을 찾아 스네이크 표기법으로 조합해줘.
    2. 데이터베이스에 정확한 정보가 없다면, 컬럼의 논리명을 분석하고 유사한 논리명을 분할하여 일치하는 물리명이 있는지 확인해. 존재한다면 그 물리명을 활용해줘.
    3. 모든 물리명은 영문 대문자이고, 스네이크 표기법이야. 절대적으로 문장으로 답변말고, 물리명만 답변해줘.
    4. 데이터베이스에서 스네이크 표기법에 따라, _는 단어의 구분자야. 질문한 단어의 수와 동일한 단어의 수로 답변해줘.
    5. 예시: 사용자가 '국가' 이라고 질문했을 때, 데이터베이스에 국가명의 물리명 'CNTR_NAME'가 존재한다면 이를 분할해서 국가의 물리명 CNTR 만 반환해줘.
    6. 예시: 사용자가 '서브'라고 입력해도, 데이터베이스에 'SUB' 컬럼이 있다면 한국어가 아닌 'SUB' 라는 실제 물리명을 그대로 답변해줘.
    ----------------------
    {context}
    ----------------------
    질문: {question}
    답변:
    """)

    rag_chain = (
        {"context": retriever, "question": RunnableLambda(lambda x: x)}
        | prompt
        | (lambda prompt_input: display_and_call_llm(prompt_input, llm))
    )

    return rag_chain.invoke(question)

# LLM 호출 시 프롬프트를 화면에 출력
def display_and_call_llm(prompt_input, llm):
    from langchain_core.messages import HumanMessage
    full_prompt = prompt_input.to_messages() if hasattr(prompt_input, 'to_messages') else [HumanMessage(content=str(prompt_input))]
    print("\n[🔍 LLM에 전달된 프롬프트]")
    for msg in full_prompt:
        print(msg.content)
    return llm.invoke(prompt_input)

# 실행 예시
if __name__ == "__main__":
    initialize_rag()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
while True:
    query = input("\n질문을 입력하세요 (종료하려면 'exit'): ")
    if query.lower() == 'exit':
        break
    result = run_query(query)
    print("\n[📌 추천 결과]")
    print(result.content)


질문을 입력하세요 (종료하려면 'exit'): llm

[🔍 LLM에 전달된 프롬프트]

    아래는 실제 데이터베이스에서 추출한 정보야. 아래 내용만 참고해서 질문에 정확하게 답변해줘.
    허구의 값을 생성하거나 문서에 없는 컬럼명을 추론하지 마.

    1. 질문이 여러 개의 단어로 구성된다면, 공백으로 분할하여 각 단어에 대응하는 컬럼(물리명)을 찾아 스네이크 표기법으로 조합해줘.
    2. 데이터베이스에 정확한 정보가 없다면, 컬럼의 논리명을 분석하고 유사한 논리명을 분할하여 일치하는 물리명이 있는지 확인해. 존재한다면 그 물리명을 활용해줘.
    3. 모든 물리명은 영문 대문자이고, 스네이크 표기법이야. 절대적으로 문장으로 답변말고, 물리명만 답변해줘.
    4. 데이터베이스에서 스네이크 표기법에 따라, _는 단어의 구분자야. 질문한 단어의 수와 동일한 단어의 수로 답변해줘.
    5. 예시: 사용자가 '국가' 이라고 질문했을 때, 데이터베이스에 국가명의 물리명 'CNTR_NAME'가 존재한다면 이를 분할해서 국가의 물리명 CNTR 만 반환해줘.
    6. 예시: 사용자가 '서브'라고 입력해도, 데이터베이스에 'SUB' 컬럼이 있다면 한국어가 아닌 'SUB' 라는 실제 물리명을 그대로 답변해줘.
    ----------------------
    [Document(id='fcbdfb2e-4f29-4f74-b914-b3e983b6c986', metadata={}, page_content='표준분류: 스마트리온 표준\n구분: 단어\n논리명: LLM\n물리명: LLM\n코드ID: None\n도메인그룹: None\n논리데이터유형: None\n물리데이터유형: None\n표준여부: Y\n승인일시: 2025-02-27\n설명: None\n\n[주의: 위 내용을 벗어난 정보를 생성하지 말 것. 실제 데이터의 컬럼명과 값만 사용할 것.]'), Document(id='d478eb72-10a1-4ee6-bc74-c5e86942bbfa', metada

KeyboardInterrupt: Interrupted by user